# PyThia

PyThia performs video prediction for simple physical systems:
given initial video frames, what will a future video frame look like?

## Technical preliminaries

We install the `pythia` package and import it.
We also import other libraries, or tools from other libraries, which will be of use.

In [1]:
pip install git+https://github.com/aremondtiedrez/pythia.git

  Cloning https://github.com/aremondtiedrez/pythia.git to /tmp/pip-req-build-oao2fr0r
  Running command git clone --filter=blob:none --quiet https://github.com/aremondtiedrez/pythia.git /tmp/pip-req-build-oao2fr0r
  Resolved https://github.com/aremondtiedrez/pythia.git to commit 47b558d8e4a48c99474aae918591306d39a1480d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pythia: filename=pythia-0.1.0-py3-none-any.whl size=107705 sha256=0b2d1d615313edea20aaa3eecf08f0e7c28237e6c3150529fafea9457589da35
  Stored in directory: /tmp/pip-ephem-wheel-cache-tmiycr9m/wheels/73/60/af/5a848e3ca8f5fdcf403911aacc928f3a3fb8578d5add2726bf
Successfully built pythia


In [2]:
import numpy as np

from IPython.display import HTML

import pythia

## IN PROGRESS

In [3]:
snapshot_timesteps, positions, velocities, images = pythia.demo.load_demo_data()

In [4]:
animation = pythia.visualization.create_animation(images[0], display_walls=True)
HTML(animation.to_jshtml())

TypeError: create_animation() got an unexpected keyword argument 'display_walls'

In [ ]:
pythia.visualization.inspect(snapshot_timesteps, positions[0], velocities[0], images[0])